In [1]:
import mat_properties as prop


import os
import time
import cotel
import GTU
import mat_properties as prop
import numpy as n
import pandas as pd
from scipy.optimize import root


# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas_KU = (
    0.710320591016015,
    0.00996710270335893,
    0.090538556815177,
    0.180531273012258,
    0.00864247645319178
)

gas_KU = prop.Materials_prop(
    gasmix,
    fractiongas_KU,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)

water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


Methane = prop.Materials_prop(
    "METHANE",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


fractionwaterMethane = (0.833372660622383, 0.166627339377617, 0, 0, 0)
waterMethanemix = "Water*METHANE"

waterMethane = prop.Materials_prop(
    waterMethanemix,
    fractionwaterMethane,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)

gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)



#Вкл выкл заряд
PKM_zaryad = False

#Доля пара на ПКМ
dolya_PKM = 0.7





if PKM_zaryad == True:
    
    from PKM import steam_transformer
    #Пар в паротрансформатор 30% от номинала!
    water_streams.loc['DROSVD-ST','T':'H'] = water_streams.loc['PEVD-DROSVD','T':'H']
    water_streams.loc['DROSVD-ST','G'] = (dolya_PKM)*water_streams.at['PEVD-DROSVD','G']
    water_streams.loc['DROSVD-TURBVD','G'] = (1-dolya_PKM)*water_streams.at['PEVD-DROSVD','G']
    
    #паротрансформатор
    ST = steam_transformer(stream11 = 'DROSVD-ST', 
                           water = water, 
                           water_streams = water_streams, 
                           heaters = heaters, 
                           Pdr1 = 2, 
                           Pdr2 = 0.8, 
                           P2 = 2, 
                           dT = 15, 
                           dTmin = 5, 
                           Tdec = 10)
    steam_trans = ST.calc()

    #Ввод в табл выходов из паротрансформатора
    water_streams.loc['ST-GPK','T':'G'] = [steam_trans['T16'],steam_trans['P16'],steam_trans['H16'],steam_trans['G1']]
    water_streams.loc['ST-PKM','T':'G'] = [steam_trans['T24'],steam_trans['P2'],steam_trans['H24'],steam_trans['G2']]
    heaters.at['Strans','Qw'] = steam_trans['Q']

    #реформер
    from PKM import reformer
    ref = reformer(stream11 = 'ST-PKM',
                   water = water,
                   gas_KU = gas_KU,
                   Methane = Methane,
                   waterMethane = waterMethane,
                   water_streams = water_streams,
                   heaters = heaters,
                   Tref = 700,
                   Pref = 2,
                   T1gas = 1968.58395330148,
                   T2gas = 800)
    reform = ref.calc()

    #Ввод в табл выходов из паротрансформатора
    syngas_streams.loc['REF-COOL','T':'G'] = [reform['Tref'],reform['Pref'],reform['Hsg'],reform['Gref']]
    heaters.at['Reformer','Qw'] = reform['Q']
    syngas_streams.loc['REF-COOL','N2':'CO'] = list(reform['SGfrac'].values())

    #Газы реформера
    gas_streams.loc['AIR-REF','T':'G']     = [15,0.1,414.38,reform['Gair']]
    gas_streams.loc['CH4-REF','T':'G']     = [15,0.7,881.50,reform['Gch4']]
    gas_streams.loc['REF-SMESH','T':'G']   = [800,0.1,reform['H2gas'],reform['Ggas']]
    gas_streams.loc['REF-SMESH','N2':'Ar'] = list(reform['Gasfrac'].values())

    #Смешение
    gas_streams.loc['GTU-PEVD','G']   = gas_streams.at['REF-SMESH','G'] + gas_streams.at['GTU-KU','G']
    gas_streams.loc['GTU-PEVD','H']   = (gas_streams.at['REF-SMESH','G']*gas_streams.at['REF-SMESH','H']+gas_streams.at['GTU-KU','G']*gas_streams.at['GTU-KU','H'])/gas_streams.loc['GTU-PEVD','G']
    gas_streams.loc['GTU-PEVD','P']   = 0.1

    #Если доли - молярные
    Sr_mol_mass1 =  (gas_streams.at['REF-SMESH','N2']*(0.0280134) +
                    gas_streams.at['REF-SMESH','O2']*(0.03199806) +
                    gas_streams.at['REF-SMESH','CO2']*(0.0440095) +
                    gas_streams.at['REF-SMESH','H2O']*(0.01801528) +
                    gas_streams.at['REF-SMESH','Ar']*(0.039948))
    Sr_mol_mass2 =  (gas_streams.at['GTU-KU','N2']*(0.0280134) +
                    gas_streams.at['GTU-KU','O2']*(0.03199806) +
                    gas_streams.at['GTU-KU','CO2']*(0.0440095) +
                    gas_streams.at['GTU-KU','H2O']*(0.01801528) +
                    gas_streams.at['GTU-KU','Ar']*(0.039948))
    mol_flow_1 = gas_streams.at['REF-SMESH','G']/Sr_mol_mass1
    mol_flow_2 = gas_streams.at['GTU-KU','G']/Sr_mol_mass2

    mol_flow1_N2 = mol_flow_1*gas_streams.at['REF-SMESH','N2']
    mol_flow1_O2 = mol_flow_1*gas_streams.at['REF-SMESH','O2']
    mol_flow1_CO2 = mol_flow_1*gas_streams.at['REF-SMESH','CO2']
    mol_flow1_H2O = mol_flow_1*gas_streams.at['REF-SMESH','H2O']
    mol_flow1_Ar = mol_flow_1*gas_streams.at['REF-SMESH','Ar']

    mol_flow2_N2 = mol_flow_2*gas_streams.at['GTU-KU','N2']
    mol_flow2_O2 = mol_flow_2*gas_streams.at['GTU-KU','O2']
    mol_flow2_CO2 = mol_flow_2*gas_streams.at['GTU-KU','CO2']
    mol_flow2_H2O = mol_flow_2*gas_streams.at['GTU-KU','H2O']
    mol_flow2_Ar = mol_flow_2*gas_streams.at['GTU-KU','Ar']

    mol_flowsumm_N2 = mol_flow1_N2 + mol_flow2_N2
    mol_flowsumm_O2 = mol_flow1_O2 + mol_flow2_O2
    mol_flowsumm_CO2 = mol_flow1_CO2 + mol_flow2_CO2
    mol_flowsumm_H2O = mol_flow1_H2O + mol_flow2_H2O
    mol_flowsumm_Ar = mol_flow1_Ar + mol_flow2_Ar
    mol_flow_summ = mol_flow_1 + mol_flow_2

    gas_streams.loc['GTU-PEVD','N2']  = mol_flowsumm_N2/mol_flow_summ
    gas_streams.loc['GTU-PEVD','O2']  = mol_flowsumm_O2/mol_flow_summ
    gas_streams.loc['GTU-PEVD','CO2'] = mol_flowsumm_CO2/mol_flow_summ
    gas_streams.loc['GTU-PEVD','H2O'] = mol_flowsumm_H2O/mol_flow_summ
    gas_streams.loc['GTU-PEVD','Ar']  = mol_flowsumm_Ar/mol_flow_summ
    
    gas_streams.loc['PEVD-IVD':'GPK-out','N2'] = gas_streams.loc['GTU-PEVD','N2']
    gas_streams.loc['PEVD-IVD':'GPK-out','O2'] = gas_streams.loc['GTU-PEVD','O2']
    gas_streams.loc['PEVD-IVD':'GPK-out','CO2'] = gas_streams.loc['GTU-PEVD','CO2']
    gas_streams.loc['PEVD-IVD':'GPK-out','H2O'] = gas_streams.loc['GTU-PEVD','H2O']
    gas_streams.loc['PEVD-IVD':'GPK-out','Ar'] = gas_streams.loc['GTU-PEVD','Ar']
    
    gas_streams.loc['PEVD-IVD':'GPK-out','G'] = gas_streams.loc['GTU-PEVD','G']
    
    gas_SMESH = prop.Materials_prop(
        gasmix,
       (gas_streams.loc['GTU-PEVD','N2'],
        gas_streams.loc['GTU-PEVD','O2'],
        gas_streams.loc['GTU-PEVD','CO2'],
        gas_streams.loc['GTU-PEVD','H2O'],
        gas_streams.loc['GTU-PEVD','Ar']),
        prop.REFPROP_h_s,
        prop.REFPROP_p_t,
        prop.REFPROP_p_h,
        prop.REFPROP_p_s,
        prop.REFPROP_p_q,
        prop.REFPROP_t_q,
        prop.REFPROP_p_rho,
        prop.REFPROP_s_q,
        RP=RP,
    )
    gas_streams.loc['GTU-PEVD','T'] = gas_SMESH.p_h(gas_streams.loc['GTU-PEVD','P'], gas_streams.loc['GTU-PEVD','H'])['T']

else:
    #Пар в паротрансформатор 30% от номинала!
    water_streams.loc['DROSVD-ST','T':'H'] = water_streams.loc['PEVD-DROSVD','T':'H']
    water_streams.loc['DROSVD-ST','G'] = 0
    water_streams.loc['DROSVD-TURBVD','G'] = water_streams.at['PEVD-DROSVD','G']
    
    gas_streams.at['GTU-PEVD','T'] = gas_streams.at['GTU-KU','T']
    gas_streams.at['GTU-PEVD','P'] = gas_streams.at['GTU-KU','P']
    gas_streams.at['GTU-PEVD','H'] = gas_streams.at['GTU-KU','H']
    gas_streams.at['GTU-PEVD','G'] = gas_streams.at['GTU-KU','G']

    gas_streams.loc['GTU-PEVD':'GPK-out','N2'] = gas_streams.loc['GTU-KU','N2']
    gas_streams.loc['GTU-PEVD':'GPK-out','O2'] = gas_streams.loc['GTU-KU','O2']
    gas_streams.loc['GTU-PEVD':'GPK-out','CO2'] = gas_streams.loc['GTU-KU','CO2']
    gas_streams.loc['GTU-PEVD':'GPK-out','H2O'] = gas_streams.loc['GTU-KU','H2O']
    gas_streams.loc['GTU-PEVD':'GPK-out','Ar'] = gas_streams.loc['GTU-KU','Ar']


gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,542.10,0.1,958.869199,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
REF-SMESH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-PEVD,542.10,0.1,958.869199,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
PEVD-IVD,468.05,0.1,500.000000,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
IVD-EVD,309.20,0.1,400.000000,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
EVD-PPND,235.90,0.1,300.000000,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
PPND-IND,233.00,0.1,200.000000,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
IND-GPK,175.50,0.1,100.000000,503.8,0.7803,0.1237,0.0301,0.0594,0.0065
GPK-out,106.90,0.1,20.000000,503.8,0.7803,0.1237,0.0301,0.0594,0.0065


In [2]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
heaters

,Qw,Qg,KPD
PEVD,NaN,NaN,NaN
IVD,NaN,NaN,NaN
EVD,NaN,NaN,NaN
PPND,NaN,NaN,NaN
IND,NaN,NaN,NaN
GPK,NaN,NaN,NaN
SP2,NaN,NaN,NaN
SP1,NaN,NaN,NaN
OD,NaN,NaN,NaN
ASW,NaN,NaN,NaN


In [4]:
water_streams

,T,P,H,G,S,X,Unnamed: 7,Unnamed: 8
AIR,15.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PEVD-DROSVD,511.500000,8.407000,3423.209978,63.400000,NaN,NaN,NaN,NaN
IVD-PEVD,301.297556,8.746000,2747.104023,63.400000,NaN,NaN,NaN,NaN
EVD-IVD,298.800000,8.746000,1337.948265,63.400000,NaN,NaN,NaN,NaN
PEN-EVD,166.500000,8.746000,708.319849,63.400000,NaN,NaN,NaN,NaN
BND-PEN,164.738361,0.696400,696.095567,63.400000,NaN,NaN,NaN,NaN
PPND-DROSND,211.600000,0.691000,2871.481837,14.760000,NaN,NaN,NaN,NaN
IND-PPND,164.738361,0.696400,2762.534980,14.760000,NaN,NaN,NaN,NaN
GPK-IND,164.600000,0.696400,695.493701,78.160000,NaN,NaN,NaN,NaN
GPK-REC,164.600000,0.696400,695.493701,82.910000,NaN,NaN,NaN,NaN
